## Exercício: Modelo de Linguagem com auto-atenção, máscaras causais e LoRA

##Priscila Marques de Oliveira
##RA094312



## Dados

Vamos usar o mesmo dataset do Machado de Assis.



In [ ]:
!git clone https://github.com/ethelbeluzzi/projetomachado

fatal: destination path 'projetomachado' already exists and is not an empty directory.


In [ ]:
import os

DATA_PATH = os.path.join("projetomachado", "textonormalizado1000.txt")

# A príncipio, não estamos limpando as linhas
with open(DATA_PATH, "r") as data_file:
    lines = [line for line in data_file]

# É possível voltar a um texto monolítico juntando as linhas.
full_data = ' '.join(lines)
full_data[:1000]

'1\n MINISTÉRIO DA CULTURA\n Fundação Biblioteca Nacional\n Departamento Nacional do Livro\n A MÃO E A LUVA\n Machado de Assis\n I\n O fim da carta\n Mas que pretendes fazer agora?\n Morrer.\n Morrer? Que idéia! Deixate disso, Estêvão. Não se morre por tão pouco...\n Morrese. Quem não padece estas dores não as pode avaliar. O golpe foi profundo, e o\n meu coração é pusilânime; por mais aborrecível que pareça a idéia da morte, pior, muito pior do\n que ela, é a de viver. Ah! tu não sabes o que isto é?\n Sei: um namoro gorado...\n Luís!\n ... E se em cada caso de namoro gorado morresse um homem, tinha já diminuído muito o\n gênero humano, e Malthus perderia o latim. Anda, sobe.\n Estêvão meteu a mão nos cabelos com um gesto de angústia; Luís Alves sacudiu a cabeça\n e sorriu. Achavamse os dois no corredor da casa de Luís Alves, à rua da Constituição,  que\n então se chamava dos Ciganos;  então, isto é, em 1853, uma bagatela de vinte anos que lá vão,\n levando talvez consigo as ilusões do

In [ ]:
# Dados já foram separados em linhas
# Checar tamanho das linhas em caracteres, por curiosidade
lines = []
line_lens = []

with open(DATA_PATH, "r") as data_file:
    for line in data_file:
        lines.append(line)
        line_lens.append(len(line))


In [ ]:
# Limpar linhas, removendo \n, espaços antes e depois
with open(DATA_PATH, "r") as data_file:
    cleaned_lines = [line.strip().lower() for line in data_file]

len(cleaned_lines)

306409

In [ ]:
sum([len(cleaned_line) for cleaned_line in cleaned_lines])

18539036

In [ ]:
# É possível voltar a um texto monolítico juntando as linhas. Nota-se que estamos adicionando espaços, mas não há mais \n
full_data = ' '.join(cleaned_lines)
len(full_data)

18845444

In [ ]:
full_data[:1000]

'1 ministério da cultura fundação biblioteca nacional departamento nacional do livro a mão e a luva machado de assis i o fim da carta mas que pretendes fazer agora? morrer. morrer? que idéia! deixate disso, estêvão. não se morre por tão pouco... morrese. quem não padece estas dores não as pode avaliar. o golpe foi profundo, e o meu coração é pusilânime; por mais aborrecível que pareça a idéia da morte, pior, muito pior do que ela, é a de viver. ah! tu não sabes o que isto é? sei: um namoro gorado... luís! ... e se em cada caso de namoro gorado morresse um homem, tinha já diminuído muito o gênero humano, e malthus perderia o latim. anda, sobe. estêvão meteu a mão nos cabelos com um gesto de angústia; luís alves sacudiu a cabeça e sorriu. achavamse os dois no corredor da casa de luís alves, à rua da constituição,  que então se chamava dos ciganos;  então, isto é, em 1853, uma bagatela de vinte anos que lá vão, levando talvez consigo as ilusões do leitor, e deixandolhe em troca usurários!

In [ ]:
# Separar em treino e teste
# Tamanhos das divisões
train_limit = int(0.7 * len(cleaned_lines))
val_limit = int(0.2 * len(cleaned_lines)) + train_limit

# Dividindo os dados
train_cleaned_lines = cleaned_lines[:train_limit]
val_cleaned_lines = cleaned_lines[train_limit:val_limit]
test_cleaned_lines = cleaned_lines[val_limit:]

# Não utilize o split val para nada a partir daqui, somente validar

In [ ]:
len(train_cleaned_lines)

214486

In [ ]:
len(val_cleaned_lines)

61281

In [ ]:
len(test_cleaned_lines)

30642

##Pré Processamento

In [ ]:
import nltk
nltk.download('punkt_tab')
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [ ]:
import re
from nltk.tokenize import word_tokenize, RegexpTokenizer

def preprocess_text(text, keep_punctuation=False):
    """
    Pré-processamento para corpus de previsão da próxima palavra.
    - lowercase
    - normaliza espaços
    - substituir números por <NUM>
    """
    text = text.lower()

    text = re.sub(r"\s+", " ", text).strip()

    tokens = word_tokenize(text)

    return tokens

In [ ]:
preprocess_trained_text = [preprocess_text(line) for line in train_cleaned_lines]

In [ ]:
train_cleaned_lines[5]

'machado de assis'

In [ ]:
preprocess_trained_text[1]

['ministério', 'da', 'cultura']

In [ ]:
len(preprocess_trained_text)

214486

In [ ]:
preprocess_val_text = [preprocess_text(line) for line in val_cleaned_lines]

In [ ]:
len(preprocess_val_text)

61281

In [ ]:
preprocess_test_text = [preprocess_text(line) for line in test_cleaned_lines]

In [ ]:
len(preprocess_test_text)

30642

In [ ]:
from collections import Counter
import re
# Contar número de palavras ÚNICAS
def count_words(texts):
    # Counter: collection especifica do Python para contar ocorrências de um objeto
    word_counts = Counter()
    for text in texts:
        word_counts.update(re.findall(r'\w+', text.lower()))
        #### Separado com regex, \w+: sequências alfanuméricas


    return word_counts

word_counts = count_words(train_cleaned_lines)


In [ ]:
word_counts

Counter({'1': 211,
         'ministério': 135,
         'da': 23790,
         'cultura': 24,
         'fundação': 18,
         'biblioteca': 165,
         'nacional': 146,
         'departamento': 11,
         'do': 26770,
         'livro': 946,
         'a': 92380,
         'mão': 1869,
         'e': 70069,
         'luva': 29,
         'machado': 478,
         'de': 75020,
         'assis': 559,
         'i': 764,
         'o': 67182,
         'fim': 1616,
         'carta': 2032,
         'mas': 16256,
         'que': 82710,
         'pretendes': 15,
         'fazer': 1571,
         'agora': 2950,
         'morrer': 461,
         'idéia': 1700,
         'deixate': 19,
         'disso': 505,
         'estêvão': 452,
         'não': 43624,
         'se': 18280,
         'morre': 101,
         'por': 11537,
         'tão': 3496,
         'pouco': 3006,
         'morrese': 7,
         'quem': 2601,
         'padece': 23,
         'estas': 871,
         'dores': 133,
         'as': 15880,

## Criando um vocabulário

In [ ]:
vocab_size = 20000
most_frequent_words = [word for word, count in word_counts.most_common(vocab_size)]
# o vocabulário irá começar a partir do 3 pois irei reservar o valores 0, 1 e 2 para os tokens especiais <unk>, <sos> e <eos> respectivamente
all_tokens = ["<UNK>", "<SOS>", "<EOS>", "<PAD>"] + most_frequent_words
vocab = {word: i for i, word in enumerate(all_tokens)}

In [ ]:
vocab

{'<UNK>': 0,
 '<SOS>': 1,
 '<EOS>': 2,
 '<PAD>': 3,
 'a': 4,
 'que': 5,
 'de': 6,
 'e': 7,
 'o': 8,
 'não': 9,
 'um': 10,
 'do': 11,
 'da': 12,
 'é': 13,
 'os': 14,
 'com': 15,
 'se': 16,
 'uma': 17,
 'em': 18,
 'mas': 19,
 'para': 20,
 'as': 21,
 'era': 22,
 'ao': 23,
 'por': 24,
 'no': 25,
 'à': 26,
 'eu': 27,
 'mais': 28,
 'na': 29,
 'como': 30,
 'lhe': 31,
 'ele': 32,
 'me': 33,
 'ou': 34,
 'foi': 35,
 'dos': 36,
 'ela': 37,
 'nem': 38,
 'disse': 39,
 'quando': 40,
 'das': 41,
 'sem': 42,
 'já': 43,
 'casa': 44,
 'meu': 45,
 'depois': 46,
 'há': 47,
 'minha': 48,
 'ser': 49,
 'tudo': 50,
 'só': 51,
 'tempo': 52,
 'tinha': 53,
 'olhos': 54,
 'nada': 55,
 'ainda': 56,
 'muito': 57,
 'd': 58,
 'também': 59,
 'outra': 60,
 'dia': 61,
 'outro': 62,
 'mesmo': 63,
 'tão': 64,
 'esta': 65,
 'seu': 66,
 'sua': 67,
 'estava': 68,
 'vez': 69,
 'até': 70,
 'porque': 71,
 'nos': 72,
 'este': 73,
 'assim': 74,
 'pouco': 75,
 'agora': 76,
 'dias': 77,
 'às': 78,
 'vida': 79,
 'aos': 80,
 'bem': 8

In [ ]:
# o tamanho total do vocab será de 10000 das palavras + 3 do tokens especiais
len(vocab)

20004

In [ ]:
vocab_size = len(vocab)

In [ ]:
#Por que deletar palavras desconhecidas?
# estou deletando a frase inteira para garantir que haja contexto que o modelo possa se basear ao fazer a predição da próxima palavra
def removeUnknownVocab(texts, vocab):
    # Remove frases que contenham palavras fora do vocabulario
    validas = []
    for sentence in texts:
        # só mantém se todas as palavras estão no vocabulário
        if all(word in vocab for word in sentence):
            validas.append(sentence)
    return validas

In [ ]:
cleaned_train_text = removeUnknownVocab(preprocess_trained_text, vocab)

In [ ]:
len(cleaned_train_text)

17126

In [ ]:
cleaned_val_text = removeUnknownVocab(preprocess_val_text, vocab)

In [ ]:
len(cleaned_val_text)

3362

In [ ]:
cleaned_test_text = removeUnknownVocab(preprocess_test_text, vocab)

In [ ]:
len(cleaned_test_text)

1611

In [ ]:
cleaned_train_text[26]


['há']

## Classe do dataset

In [ ]:
def tokenizer(sentence, vocab):
    #print(sentence)
    if isinstance(sentence, str):
        words = [sentence]  # trata como palavra inteira
    else:
        words = sentence  # já é lista de palavras

    return [vocab.get(word, vocab["<UNK>"]) for word in words] # 0 for OOV

In [ ]:
idx_to_word = {i: w for w, i in vocab.items()}

In [ ]:
#def detokenizer(tokens):
    #return [idx_to_word[tok] for tok in tokens] # 0 for OOV

In [ ]:
def detokenizer(tokens):
    return [
        "<PAD>" if tok == -100 else idx_to_word[tok]
        for tok in tokens
    ]

In [ ]:
cleaned_train_text[3]

['departamento', 'nacional', 'do', 'livro']

In [ ]:
teste = tokenizer(cleaned_train_text[3], vocab)
teste

[12221, 1555, 11, 265]

In [ ]:
det = detokenizer(teste)
det

['departamento', 'nacional', 'do', 'livro']

In [ ]:
cleaned_train_text[:10]

[['1'],
 ['ministério', 'da', 'cultura'],
 ['fundação', 'biblioteca', 'nacional'],
 ['departamento', 'nacional', 'do', 'livro'],
 ['a', 'mão', 'e', 'a', 'luva'],
 ['machado', 'de', 'assis'],
 ['i'],
 ['o', 'fim', 'da', 'carta'],
 ['um', 'so'],
 ['o', 'era', 'mau']]

In [ ]:
train_text_tokenized = []
train_text_tokenized.extend(["<SOS>"]+line+["<EOS>"] for line in cleaned_train_text)
val_text_tokenized = []
val_text_tokenized.extend(["<SOS>"]+line+["<EOS>"] for line in cleaned_val_text)
test_text_tokenized = []
test_text_tokenized.extend(["<SOS>"]+line+["<EOS>"] for line in cleaned_test_text)

In [ ]:
train_text_tokenized[:10]

[['<SOS>', '1', '<EOS>'],
 ['<SOS>', 'ministério', 'da', 'cultura', '<EOS>'],
 ['<SOS>', 'fundação', 'biblioteca', 'nacional', '<EOS>'],
 ['<SOS>', 'departamento', 'nacional', 'do', 'livro', '<EOS>'],
 ['<SOS>', 'a', 'mão', 'e', 'a', 'luva', '<EOS>'],
 ['<SOS>', 'machado', 'de', 'assis', '<EOS>'],
 ['<SOS>', 'i', '<EOS>'],
 ['<SOS>', 'o', 'fim', 'da', 'carta', '<EOS>'],
 ['<SOS>', 'um', 'so', '<EOS>'],
 ['<SOS>', 'o', 'era', 'mau', '<EOS>']]

In [ ]:
palavra = ['<SOS>', 'departamento', 'nacional', 'do', 'livro', '<EOS>', '<pad>']
teste = tokenizer(palavra, vocab)
teste

[1, 12221, 1555, 11, 265, 2, 0]

In [ ]:
context_size = 9 # 9 palavras de entrada. O target é a próxima palavra

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader


# MELHORAR CÓDIGO
# CÓDIGO INICIAL, MAS FUNCIONA
class MachadaoDataset(Dataset):
    def __init__(self, vocab, context_size, data):
        self.vocab = vocab
        self.context_size = context_size
        self.data = []

        context_ids = []
        target_ids = []


        for line in data:
            indice = 0

            # em minha defesa não existe do-while no python, não que fazer do-while seja mais bonito, mas entre do-while e while-true a resposta é meio obvia
            while True:

                context = line[indice: indice + context_size]
                target = line[indice + 1:  (indice + 1) + context_size]

                if (len(context) < context_size):
                    context = context + (["<PAD>"] * (context_size - (len(context))))

                if (len(target) < context_size):
                    target = target + (["<PAD>"] * (context_size - (len(target))))

                indice = indice + context_size

                if ("<EOS>" in target):
                    break

            context_ids.append(tokenizer(context, self.vocab))
            # substitui 0 e 3 por -100 no target
            target_token_ids = tokenizer(target, self.vocab)
            target_token_ids = [-100 if t in (0, 3) else t for t in target_token_ids]
            target_ids.append(target_token_ids)

        self.X = torch.tensor(context_ids, dtype=torch.long)
        self.y = torch.tensor(target_ids, dtype=torch.long)
    def __len__(self):
      return len(self.X)

    def __getitem__(self, idx):
      return self.X[idx], self.y[idx]

In [ ]:
train_dataset = MachadaoDataset(vocab, context_size, train_text_tokenized)
val_dataset = MachadaoDataset(vocab, context_size, val_text_tokenized)
test_dataset = MachadaoDataset(vocab, context_size, test_text_tokenized)

In [ ]:
train_dataset[26]

(tensor([ 1, 47,  2,  3,  3,  3,  3,  3,  3]),
 tensor([  47,    2, -100, -100, -100, -100, -100, -100, -100]))

In [ ]:
train_dataset[26][0]

tensor([ 1, 47,  2,  3,  3,  3,  3,  3,  3])

In [ ]:
train_dataset[26][0].tolist()

[1, 47, 2, 3, 3, 3, 3, 3, 3]

In [ ]:
lin1 = detokenizer(train_dataset[26][0].tolist())
lin1

['<SOS>', 'há', '<EOS>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>']

In [ ]:
lin2 = detokenizer(train_dataset[26][1].tolist())
lin2

['há', '<EOS>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>']

In [ ]:
len(train_dataset)

17126

In [ ]:
len(val_dataset)

3362

In [ ]:
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
sample = next(iter(train_loader))

## Model

In [ ]:
import torch.nn as nn
import torch.nn.functional as F
import math

In [ ]:
# code based in https://saurabhraj5162.medium.com/day-5-lora-from-scratch-using-pytorch-ai-ml-coding-series-c28e12c39f47
# a base do LoRA seria W_merged = W + (alpha/rank) * (B*(A*x))

class LoraLinear(nn.Module):
    def __init__(self, in_features, out_features, rank, alpha, dropout):
        super().__init__()
        self.rank = rank
        self.alpha = alpha
        self.in_features = in_features
        self.out_features = out_features
        self.dropout = nn.Dropout(dropout)

        # Usa o Parameter para informar que este tensor será treinavel e aparecerá nos parametros, terá gradientes atualizados e será atualizado pelo otimizador
        # Por que A usa in_featues e B usa out_features?
        self.A = nn.Parameter(torch.zeros(self.rank, self.in_features))
        self.B = nn.Parameter(torch.zeros(self.out_features, self.rank))

        # Coloca valores aleatórios para A segundo a distribuição de Kaiming/He
        # Ajuda  evitar gradientes explosivos ou vanishing
        # a = math.sqrt(5) para uma escala parecida com nn.Linear e manter a compatibilidade entre pesos LoRA e pesos padrão
        nn.init.kaiming_uniform_(self.A, a = math.sqrt(5))
        # B é inicializada com 0
        nn.init.zeros_(self.B)

        self.weight = torch.empty(self.out_features, self.in_features)


    def forward(self, original_weight, embedding):
        # Pega os pesos que da camada com X
        # usa nn.functional.linear para pegar os pesos, ao inves de criar uma nn.Linear
        # Se assemelha a x * Wt + b
        # Não faz sentido pegar apenas o peso original, é necessário usar a saída que seria do Linear
        self.weight = original_weight.data

        weight = F.linear(embedding, self.weight)

        scaling = self.alpha/self.rank

        dropout = self.dropout(embedding)

        # usamos matmul, por que é a multiplicação classica de matrizes linhaXcoluna
        A_weight = torch.matmul(dropout, self.A.t())
        BA = torch.matmul(A_weight, self.B.t())

        # (alpha/rank) * (BAx)
        scaled_lora = scaling*BA

        # Soma a camada LoRA calculada aos pesos do modelo
        lora = weight + scaled_lora

        return lora




In [ ]:
class AttentionHead(torch.nn.Module):
    def __init__(self, embedding_dim, head_dim, context_size, rank, alpha, dropout=0.3):
        super().__init__()

        # embedding_dim é o tamanho total do meu embedding
        # head_dim é meu embedding/head_count, ou seja, o tamanho do meu embeedings dividio pela quantidade de cabeças de atenção
        # essa divisão ajuda na hora do calulo da matriz
        self.Wq = nn.Linear(embedding_dim, head_dim, bias=False)
        self.Wk = nn.Linear(embedding_dim, head_dim, bias=False)
        self.Wv = nn.Linear(embedding_dim, head_dim, bias=False)

        self.Wq_lora = LoraLinear(self.Wq.in_features, self.Wq.out_features, rank, alpha, dropout)
        self.Wv_lora = LoraLinear(self.Wv.in_features, self.Wv.out_features, rank, alpha, dropout)

        # torch.tril cria uma matrix de 1s onde apenas a diagonal inferior receberá valores, o resto será 0s
        # self.register_buffer registra o elemento como parte da classe, mas não como parametro treinavel, não srecebe gradientte e nem será atualizado com otimizador
        self.register_buffer('tril', torch.tril(torch.ones(context_size, context_size)))

        # Dropout desliga alguns neuronios para evitar overfiting e tornar o modelo mais generalista
        self.dropout = nn.Dropout(0.1)

    def forward(self, embedding, has_lora=False):
        batch_size, context_size, embed_dim = embedding.size() # (batch_size, seq_len, emb_dim)

        if(has_lora):
            Q = self.Wq_lora(self.Wq.weight, embedding)
            V = self.Wv_lora(self.Wv.weight, embedding)
        else:
            Q = self.Wq(embedding) # (batch_size, seq_len, head_dim)
            V = self.Wv(embedding)

        K = self.Wk(embedding)

        scores = torch.matmul(Q, K.transpose(-2, -1)) / (K.size(-1) ** 0.5)  # (batch, seq_len, seq_len)

        # Calculo a mascara causal
        # Tudo no triangulo de cima recebe -inf, e fica "desligado" para os calculos
        causal_mask = scores.masked_fill(self.tril[:context_size, :context_size] == 0, float('-inf')) # (batch, seq_len, seq_len)

        probs = F.softmax(causal_mask, dim=-1)  # (batch, seq_len, seq_len)

        drop = self.dropout(probs)

        outputs = torch.matmul(drop, V)  # (batch, seq_len, head_dim)

        return outputs

In [ ]:
class MultiHeadAttention(torch.nn.Module):
    def __init__(self, embedding_dim, head_number, context_size, rank, alpha, dropout=0.3):
        super().__init__()

        # Divide pelo numero de cabeças, para que na hora da concatenação, não dê erro de shape
        self.emb_dim = embedding_dim // head_number

        # Cria uma lista com a quantidade de cabeças de atenção que serão uilizadas
        self.heads = nn.ModuleList(
            [AttentionHead(embedding_dim, self.emb_dim, context_size, rank, alpha) for _ in range(head_number)]
        )

        # camada de projeção e dropout
        self.Wout = nn.Linear(embedding_dim, embedding_dim)
        self.Wout_lora = LoraLinear(self.Wout.in_features, self.Wout.out_features, rank, alpha, dropout)
        self.dropout = nn.Dropout(dropout)

    def forward(self, embedding, has_lora=False):
        # faz a concatenação do retorno cabeça de atenção
        # por causa do dim=-1, a concatenação será apenas na ultima dimensão
        # o shape retornado por cada cabeça é (batch, seq_len, embedding_dim // head_number)
        # ao fazer a concatenação o shape final será (batch, seq_len, embedding_dim)
        concat = torch.cat([head(embedding, has_lora) for head in self.heads], dim=-1)

        if (has_lora):
            w_out = self.Wout_lora(self.Wout.weight, concat)
        else:
            w_out = self.Wout(concat)

        drop = self.dropout(w_out)
        return drop

In [ ]:
class TransformerBlock(torch.nn.Module):
    def __init__(self, embedding_dim, head_number, context_size, rank, alpha, dropout=0.3):
        super().__init__()

        self.multi_head = MultiHeadAttention(embedding_dim, head_number, context_size, rank, alpha)

        # MLP
        self.linear1 = nn.Linear(embedding_dim, 4 * embedding_dim)
        self.linear1_lora = LoraLinear(self.linear1.in_features, self.linear1.out_features, rank, alpha, dropout)
        self.relu = nn.ReLU()
        self.linear2 = nn.Linear(4 * embedding_dim, embedding_dim)
        self.linear2_lora = LoraLinear(self.linear2.in_features, self.linear2.out_features, rank, alpha, dropout)
        self.dropout = nn.Dropout(dropout)

        # Camadas de normalização
        self.layer_norm1 = nn.LayerNorm(embedding_dim)
        self.layer_norm2 = nn.LayerNorm(embedding_dim)

    def forward(self, embedding, has_lora=False):

        layer_norm1 = self.layer_norm1(embedding) # (batch_size, context_size, embedding_dim)
        multi_head = self.multi_head(layer_norm1, has_lora) # (batch_size, context_size, embedding_dim)
        embedding = embedding + multi_head # (batch_size, context_size, embedding_dim)


        layer_norm2 = self.layer_norm2(embedding) # (batch_size, context_size, embedding_dim)

        if (has_lora):
            linear1 = self.linear1_lora(self.linear1.weight, layer_norm2) # (batch_size, context_size, 4 * embedding_dim)
            relu = self.relu(linear1)
            linear2 = self.linear2_lora(self.linear2.weight, relu)
        else:
            linear1 = self.linear1(layer_norm2) # (batch_size, context_size, 4 * embedding_dim)
            relu = self.relu(linear1)
            linear2 = self.linear2(relu)


        embedding = embedding + linear2

        return embedding

In [ ]:
class LanguageModel(torch.nn.Module):
    def __init__(self, vocab_size, embedding_dim, head_number, context_size, layer_number, rank, alpha, dropout=0.3):
        super().__init__()
        """TODO: implementar o modelo de linguagem"""
        # Camada de embeddings
        # vocab_size = tamanho do vocabulario
        # embedding_dim = representação vetorial de cada palavra
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)

        #Cria um vetor com o tamanho do vetor de embedding e uma quantidade de linhas
        self.pos_embeddings = nn.Embedding(context_size, embedding_dim)


        # O * desempacota e envia os objetos de forma separada
        # Sequential é conteiner do python que encaixa modulos em sequencia
        self.transformer_blocks = nn.ModuleList(
                                        [TransformerBlock(embedding_dim, head_number, context_size, rank, alpha)
                                                                  for _ in range(layer_number)]
                                                )

        self.layer_norm = nn.LayerNorm(embedding_dim)
        self.linear_out = nn.Linear(embedding_dim, vocab_size)

    def forward(self, inputs, targets=None, has_lora=False):

        batch_size, seq_len = inputs.size()

        embed = self.embeddings(inputs) # (batch_size, context_size, embedding_dim)

        positions = torch.arange(seq_len, device=inputs.device).unsqueeze(0).expand(batch_size, seq_len)
        pos_emb = self.pos_embeddings(positions) # (batch_size, context_size, embedding_dim)

        embeddings = embed + pos_emb # (batch_size, context_size, embedding_dim)

        emb_transformer = embeddings
        for block in self.transformer_blocks:
            emb_transformer = block(emb_transformer, has_lora)
        transformer_block = emb_transformer

        layer_norm = self.layer_norm(transformer_block) # (B,T,C)

        logits = self.linear_out(layer_norm)

        return logits

In [ ]:
model = LanguageModel(vocab_size, 128, 4, context_size, 4, rank=4, alpha=32)

In [ ]:
# sample = next(iter(train_loader))
input = sample[0]
target = sample[1]

In [ ]:
input

tensor([[    1,   112,  6757,     2,     3,     3,     3,     3,     3],
        [    1,  8243,     2,     3,     3,     3,     3,     3,     3],
        [    1,  2291,     2,     3,     3,     3,     3,     3,     3],
        [    1,     8, 12093,     2,     3,     3,     3,     3,     3],
        [    1,   961,     2,     3,     3,     3,     3,     3,     3],
        [    1,  3657,     2,     3,     3,     3,     3,     3,     3],
        [    1,  1554,   683,     2,     3,     3,     3,     3,     3],
        [    1,  1521,     2,     3,     3,     3,     3,     3,     3],
        [    1,     9,    33,  9479,    55,     5,   292,  2794,  5488],
        [ 1057,    11,    88,     5,    37,    31,     2,     3,     3],
        [    1,  1763,     2,     3,     3,     3,     3,     3,     3],
        [    1,  8444,     2,     3,     3,     3,     3,     3,     3],
        [ 1006,  3232,     4,   152,  3655,     2,     3,     3,     3],
        [    1,  2263, 12397,     2,     3,     3, 

In [ ]:
logits = model(input, target, False)

In [ ]:
"""for name, p in model_teste.named_parameters():
    if "A" in name or "B" in name:
        p.requires_grad = True
    else:
        p.requires_grad = False"""

'for name, p in model_teste.named_parameters():\n    if "A" in name or "B" in name:\n        p.requires_grad = True\n    else:\n        p.requires_grad = False'

In [ ]:
#logits = model_teste(input, target, True)

In [ ]:
logits.shape

torch.Size([32, 9, 20004])

In [ ]:
logits.argmax(dim=1)

tensor([[2, 6, 6,  ..., 7, 3, 1],
        [5, 6, 1,  ..., 7, 1, 1],
        [5, 6, 1,  ..., 7, 0, 7],
        ...,
        [5, 6, 1,  ..., 7, 1, 3],
        [0, 6, 1,  ..., 7, 0, 5],
        [6, 3, 6,  ..., 7, 2, 3]])

In [ ]:
target

tensor([[  112,  6757,     2,  -100,  -100,  -100,  -100,  -100,  -100],
        [ 8243,     2,  -100,  -100,  -100,  -100,  -100,  -100,  -100],
        [ 2291,     2,  -100,  -100,  -100,  -100,  -100,  -100,  -100],
        [    8, 12093,     2,  -100,  -100,  -100,  -100,  -100,  -100],
        [  961,     2,  -100,  -100,  -100,  -100,  -100,  -100,  -100],
        [ 3657,     2,  -100,  -100,  -100,  -100,  -100,  -100,  -100],
        [ 1554,   683,     2,  -100,  -100,  -100,  -100,  -100,  -100],
        [ 1521,     2,  -100,  -100,  -100,  -100,  -100,  -100,  -100],
        [    9,    33,  9479,    55,     5,   292,  2794,  5488,     2],
        [   11,    88,     5,    37,    31,     2,  -100,  -100,  -100],
        [ 1763,     2,  -100,  -100,  -100,  -100,  -100,  -100,  -100],
        [ 8444,     2,  -100,  -100,  -100,  -100,  -100,  -100,  -100],
        [ 3232,     4,   152,  3655,     2,  -100,  -100,  -100,  -100],
        [ 2263, 12397,     2,  -100,  -100,  -100, 

In [ ]:
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
non_trainable_params = sum(p.numel() for p in model.parameters() if not p.requires_grad)

print(f"Parâmetros treináveis: {trainable_params:,}")
print(f"Parâmetros não-treináveis: {non_trainable_params:,}")
print(f"Total: {trainable_params + non_trainable_params:,}")

Parâmetros treináveis: 5,979,044
Parâmetros não-treináveis: 0
Total: 5,979,044


## Training

In [ ]:
# Verifica se há uma GPU disponível e define o dispositivo para GPU se possível, caso contrário, usa a CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [ ]:
import math

def MeasurePerplexity(model, data_loader, criterion, device, has_lora=False):
    model.eval()

    total_loss = 0.0
    total_samples = 0

    with torch.no_grad():
        for inputs, labels in data_loader:
          inputs = inputs.to(device)
          labels = labels.to(device)

          outputs = model(inputs, labels, has_lora)

          batch_size, seq_len, vocab_size = outputs.shape
          outputs = outputs.view(batch_size*seq_len, vocab_size)  # (batch_size*seq_len, vocab_size)
          labels = labels.view(batch_size*seq_len)   # (batch_size*seq_len)

          # SUGESTÃO PARA IGNORAR OS UNK loss = F.cross_entropy(logits, targets, ingnore_index=[...])
          # colocar o valor token <unk> no ignore_index
          loss = criterion(outputs, labels)

          batch_size = inputs.size(0)
          total_loss += loss.detach() * batch_size
          total_samples += batch_size

        avg_loss = total_loss / total_samples
        perplexity = torch.exp(avg_loss).item()

        return avg_loss.item(), perplexity

In [ ]:
import time
import random
import torch.optim as optim

random.seed(42)
epochs = 10
lr = 5e-5

model = model.to(device)
# CrossEntropy quantifica o quão bem as predições do modelo se igualam aos resultado reais
# Quanto maior confiança o modelo tem em predizer corretamente, menor a loss
# Quanto maior a confiança do modelo me predizer errado, maior a loss
#criterion = nn.CrossEntropyLoss(ignore_index=vocab["<PAD>"])
criterion = nn.CrossEntropyLoss()

optimizer = optim.AdamW(model.parameters(), lr=lr, weight_decay=0.01)

In [ ]:
"""loss, perp = MeasurePerplexity(model, val_loader, criterion, device)

print("Dados do modelo antes do treinamento")
print("Perplexity: ", perp)
print("Loss: ", loss)"""

'loss, perp = MeasurePerplexity(model, val_loader, criterion, device)\n\nprint("Dados do modelo antes do treinamento")\nprint("Perplexity: ", perp)\nprint("Loss: ", loss)'

Treinamento com modelo sem LoRA - 70% do dataset

In [ ]:
epochs = 10

for epoch in range(epochs):
    start_time = time.time()  # Start time of the epoch
    epoch_loss = 0
    total_samples = 0
    model.train()
    for inputs, labels in train_loader:

        inputs = inputs.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(inputs, labels, False)

        batch_size, seq_len, vocab_size = outputs.shape
        outputs = outputs.view(batch_size*seq_len, vocab_size)  # (batch_size*seq_len, vocab_size)
        labels = labels.view(batch_size*seq_len)   # (batch_size*seq_len)

        loss = criterion(outputs, labels)

        # Backward
        # Zerando os gradiente calculados
        optimizer.zero_grad()
        # Fazendo calculo de backpropagation
        loss.backward()
        # Atualizando os pesos do modelo
        optimizer.step()

        epoch_loss+= loss.detach() * inputs.size(0)
        total_samples += inputs.size(0)

    end_time = time.time()  # End time of the epoch
    epoch_duration = end_time - start_time  # Duration of epoch
    avg_epoch_loss = epoch_loss / total_samples
    train_perplexity = torch.exp(avg_epoch_loss).item()

    print('Training Data:')
    print(f'Epoch [{epoch+1}/{epochs}], \
            Loss: {avg_epoch_loss.item():.4f},\
            Perplexity: {train_perplexity:.4f},\
            Elapsed Time: {epoch_duration:.2f} sec')

Training Data:
Epoch [1/10],             Loss: 7.1457,            Perplexity: 1268.5948,            Elapsed Time: 13.03 sec
Training Data:
Epoch [2/10],             Loss: 5.7257,            Perplexity: 306.6556,            Elapsed Time: 13.05 sec
Training Data:
Epoch [3/10],             Loss: 5.4689,            Perplexity: 237.2003,            Elapsed Time: 14.31 sec
Training Data:
Epoch [4/10],             Loss: 5.3130,            Perplexity: 202.9600,            Elapsed Time: 12.78 sec
Training Data:
Epoch [5/10],             Loss: 5.1909,            Perplexity: 179.6364,            Elapsed Time: 12.90 sec
Training Data:
Epoch [6/10],             Loss: 5.0804,            Perplexity: 160.8309,            Elapsed Time: 12.91 sec
Training Data:
Epoch [7/10],             Loss: 4.9848,            Perplexity: 146.1788,            Elapsed Time: 13.03 sec
Training Data:
Epoch [8/10],             Loss: 4.8945,            Perplexity: 133.5466,            Elapsed Time: 12.78 sec
Training Data:


Treinamento do modelo congelando os pesos e treinando somente A e B - 20% do dataset

In [ ]:
for name, p in model.named_parameters():
    if "A" in name or "B" in name:
        p.requires_grad = True
    else:
        p.requires_grad = False

In [ ]:
optimizer = optim.AdamW(model.parameters(), lr=lr, weight_decay=0.01)

In [ ]:
epochs = 10

for epoch in range(epochs):
    start_time = time.time()  # Start time of the epoch
    epoch_loss = 0
    total_samples = 0
    model.train()
    for inputs, labels in val_loader:

        inputs = inputs.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(inputs, labels, True)

        batch_size, seq_len, vocab_size = outputs.shape
        outputs = outputs.view(batch_size*seq_len, vocab_size)  # (batch_size*seq_len, vocab_size)
        labels = labels.view(batch_size*seq_len)   # (batch_size*seq_len)

        loss = criterion(outputs, labels)

        # Backward
        # Zerando os gradiente calculados
        optimizer.zero_grad()
        # Fazendo calculo de backpropagation
        loss.backward()
        # Atualizando os pesos do modelo
        optimizer.step()

        epoch_loss+= loss.detach() * inputs.size(0)
        total_samples += inputs.size(0)

    end_time = time.time()  # End time of the epoch
    epoch_duration = end_time - start_time  # Duration of epoch
    avg_epoch_loss = epoch_loss / total_samples
    train_perplexity = torch.exp(avg_epoch_loss).item()

    print('Training Data:')
    print(f'Epoch [{epoch+1}/{epochs}], \
            Loss: {avg_epoch_loss.item():.4f},\
            Perplexity: {train_perplexity:.4f},\
            Elapsed Time: {epoch_duration:.2f} sec')

Training Data:
Epoch [1/10],             Loss: 5.4997,            Perplexity: 244.6268,            Elapsed Time: 3.68 sec
Training Data:
Epoch [2/10],             Loss: 5.4463,            Perplexity: 231.8893,            Elapsed Time: 4.26 sec
Training Data:
Epoch [3/10],             Loss: 5.3926,            Perplexity: 219.7722,            Elapsed Time: 3.62 sec
Training Data:
Epoch [4/10],             Loss: 5.3695,            Perplexity: 214.7598,            Elapsed Time: 3.62 sec
Training Data:
Epoch [5/10],             Loss: 5.3493,            Perplexity: 210.4600,            Elapsed Time: 4.28 sec
Training Data:
Epoch [6/10],             Loss: 5.3353,            Perplexity: 207.5376,            Elapsed Time: 3.59 sec
Training Data:
Epoch [7/10],             Loss: 5.3258,            Perplexity: 205.5816,            Elapsed Time: 3.63 sec
Training Data:
Epoch [8/10],             Loss: 5.3147,            Perplexity: 203.3059,            Elapsed Time: 4.80 sec
Training Data:
Epoch [9/

Teste do modelo - 10% do dataset

In [ ]:
loss, perp = MeasurePerplexity(model, test_loader, criterion, device, True)

print("Validation Data:")
print("Perplexity: ", perp)
print("Loss: ", loss)

Validation Data:
Perplexity:  147.2348175048828
Loss:  4.992028713226318


## Exemplo de uso

In [ ]:
import torch
import torch.nn.functional as F

def sample_next_token(logits, top_k=50, top_p=0.9, temperature=1.0):
    # Ajusta a "temperatura" para controlar a aleatoriedade
    # temperatura = 1 sugere um comportamento normal
    # Se dividir por um valor < 1 os logits ficam maiores, fazendo com que os maiores se tornem mais expressivos
    # se dividir por um valor > 1, penalizo os logits menores, que ficaram menores ainda
    logits = logits / temperature
    # converte me probabilidades não negativas
    probs = F.softmax(logits, dim=-1)

    # top_k representa a quantidade de logits mais representattivos que será uttilizada
    if top_k > 0:
        # retorna os os valores mais altos e seus ids
        topk_probs, topk_idx = torch.topk(probs, top_k)
        # cria uma mascara booleana
        mask = torch.ones_like(probs, dtype=torch.bool)
        # pcoloca False nos indidce que não fazem parte da seleção top_k
        mask.scatter_(0, topk_idx, False)
        # Zera as probs de todos os ids fora das prob de top_k
        probs = probs.masked_fill(mask, 0)

    # ----- TOP-P (nucleus) -----
    if top_p < 1.0:
        # ordena os topk do maior para o menor
        sorted_probs, sorted_idx = torch.sort(probs, descending=True)
        cumulative_probs = torch.cumsum(sorted_probs, dim=-1)
        # marca o elemento que soma seja mairo que top_p
        mask = cumulative_probs > top_p
        # este código é para marcar todos os valores q são maiores que o top_p, o primeiro q ultrapassou, fica na lista
        mask[1:] = mask[:-1].clone()  # shift
        mask[0] = False
        # Zera a problabilidade de todos os outros
        probs = probs.clone()
        # volta o probs a oderm normal
        probs.scatter_(0, sorted_idx[mask], 0)

    # normaliza de novo para q soma de 1
    probs = probs / probs.sum()
    # Amostra uma palavra do vetor de probabilidades
    next_token = torch.multinomial(probs, 1)
    return next_token.item()


In [ ]:
def generate_text(text, max_length, context_size):
    """TODO: implemente a função para gerar texto até atingir o max_length"""
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)
    model.eval()

    words = text.split()
    words_ids = tokenizer(words[len(words) - context_size : len(words)], vocab)


    # Verifica se o texto tem a quantidade minima de palavras para a execução do modelo
    with torch.no_grad():
        # Continua executando ate atingir o maximo de palavras
        while len(words_ids) < max_length:
            # Caso a sentenção não tenha o minimo de palavras de contexto, não faz a execução
            if len(words_ids) < context_size:
                break

            work_id = words_ids[len(words_ids) - context_size : len(words_ids)]

            words_tensor = torch.tensor(work_id, dtype=torch.long).unsqueeze(0).to(device)

            output = model(words_tensor)

            last_logits = output[:, -1, :] # (1, vocab_size)
            last_logits = logits[0, -1] # (vocab_size)

            #prob_word = sample_next_word(last_logits[0], 1.0, 50)
            prob_word = sample_next_token(last_logits, top_k=50, top_p=0.9, temperature=0.8)

            if prob_word == vocab["<EOS>"]:
                break  # fim da sequência

            words_ids.append(prob_word)


    words = detokenizer(words_ids)

    return ' '.join(words)

In [ ]:
context = 9
max_length= 100
text = "<SOS> valia a pena admirar como eles comunicavam a"
retorno = generate_text(text, max_length, context)
print(retorno)

<SOS> valia a pena admirar como eles comunicavam a chupado cuidavam franziu seguro patimau empuxar gênios anacoreta involuntário anacoreta perversa franziu mon fatídico mem votou franziu humanas despregadas teremos estejam maliciosamente estejam caboclo maliciosamente trajando versões suceder fatídico batem votou teremos lâmpada fatídico versões coberto alento seguro agudeza perversa incumbido suceder seguro versões mandam pedidos empuxar seguro perguntei confrades infelicidade estejam gênios maliciosamente perguntei coberto lâmpada lâmpada franziu caboclo caboclo lâmpada agudeza perversa fatídico lisa mandam lâmpada cinzenta perguntei empuxar estejam ombro incumbido achavaa chupado mon pernas agudeza os involuntário gênios patimau cabo teremos chupado cuidavam pedidos duvidou duvidou votou
